In [ ]:
from common import *
from search.imports import *
from search.search import *

### making test variables

In [ ]:
from scipy.stats import norm
ppf = norm.ppf
eps = np.finfo(float).eps

In [ ]:
def make_new_vars():
    n = 1000
    ##################RANDOMIZE#######################
    while True:
        try:
            u = np.random.uniform(size=(5, n))
            assert (u!=0).all()
            break
        except AssertionError:
            continue
    uu = u.copy()
    uu[0][np.random.randint(252, size=n) == 0] = np.nan
    uu[1][np.random.randint(173, size=n) == 0] = np.nan
    uu[2][np.random.randint(81, size=n) == 0] = np.nan
    uu[3][np.random.randint(27, size=n) == 0] = np.nan
    #################ENDRANDOM######################
    F = pd.DataFrame({'hax': np.arange(n)})
    F['time'] = F.hax // 10
    F['assetCodeId'] = F.hax % 10
    F['quarter'] = F.hax // 100
    P = F.copy()
    F['alex'] = ppf(u[0]*5%.99+eps)*ppf(u[1]*7%.99+eps)
    F['bob'] = ppf(.5+.49*uu[2]*np.sin(F.assetCodeId.values**10))*np.exp(uu[3])
    F['carol'] = -np.log(u[0]) + 4 * np.sin(1/uu[1])
    F['dean'] = u[1] + 2 * uu[2] - np.exp(-uu[3])
    F['edgar'] = ppf( (uu[2]**2-2*u[1]+(u[0]-u[3])**3-.5*u[1]**2) % .99 + eps )
    P['y'] = (ppf((u[0]+u[1])/2) + ppf((u[2]+u[3])/2) + .2*ppf(u[4])) * 1e-1
    P['universe'] = (~np.isnan(uu[0])).astype(float)
    return F, P

In [ ]:
#F, P = make_new_vars()
#Path('lighttest.pkl').write_bytes(pickle.dumps((F, P)))
F, P = pickle.loads((top_dir/'search/lighttest.pkl').read_bytes())

In [ ]:
P['target'] = P.y>0
P['upDown'] = (P.target*2-1)
P['upDown1'] = P.upDown*P.universe.astype(int)
P['absVal'] = np.abs(P.y)
P['absVal1'] = P.absVal*P.universe
P['weight'] = P.absVal#.qtl()
P['weight1'] = P.weight*P.universe

In [ ]:
F.head()

In [ ]:
P.head()

In [2]:
class LSP(O()):
    class Discrete(O()):
        stop = {1: 1., 2: 2.2}
        obo = {0: 0}
        enc = {
            'learning_rate': [.05],
            ('max_depth','num_leaves'):
                [(3,1<<3),(6,1<<6),(-1,1<<9)]
        }
        
    class OneByOne(O()):
        class info(O()):
            a = "main [a]rray data of 3 values, [min mid max]"
            b =  "[b]ack up value, i.e. default value if array doesn't give better results"
            cast = "function to apply to values before using to cast to the right dtype"
            lim = "maximum number of iterations of searching in this hyperparameter"
        class default(O()):
            cast = keepSigFig(2)
            lim = 0
        enc = {
            0: {
                'min_data_in_leaf': O(a=[1,6,37], cast=round, lim=lambda lim: lim+0),
                'min_sum_hessian_in_leaf': O(a=[0,5,20], b=0),
                'lambda_l1': O(a=[0,.02,.2], b=0),
                'lambda_l2': O(a=[0,.02,.2], b=0),
            }
        }


class LSM(O()):
    model = O(
        time = 'time',
        value = 'y',
        target = 'target',
        weight = 'weight1',
    )
    
    metric = KaggleMetric()
    search = LPS
    
    num_feats = 3
    num_samps = 10
    
@inside(LSM)
class Step(metaclass=staticclass):
    best = -float('inf')
    stop = {1: 1.5, 2: 3., 3: float('inf')}
    def func(results, *, t):
        t += 1
        self, stop = __class__, __class__.stop
        num_feats, num_samps = LSM.num_feats, LSM.num_samps

        if results is not None and results['score'] > self.best:
            self.best = results['score']

        ts = t % num_samps
        if ts == 0: # if new feature, reset `self.best` score
            self.best = -float('inf')

        if ts in stop and stop[ts] > self.best:
            t = (t//num_samps + 1) * num_samps # go to next feature if met stop score criterion

        if t >= num_feats * num_samps:
            return None
        return t
    
@inside(LSM)
class Features(metaclass=staticclass):
    '''features selection groups'''
    data = [
        ['alex', 'bob', 'carol'],
        ['bob', 'dean', 'edgar'],
        ['alex', 'carol', 'edgar']
    ]
    def func(*, t):
        num_feats, num_samps = LSM.num_feats, LSM.num_samps
        return O(Feats=__class__.data[t // num_samps])

ho = P.quarter >= 8

@inside(LSM)
class Samples(metaclass=staticclass):
    '''sample learning/cv split'''
#         enc = [
#             O(method='group.2', groups='quarter', data=[
#                 [0, 1, 2, 3, 4],
#                 [5, 6, 7, 8, 9]
#             ]),
#         ]
    # O(method='GroupShuffleSplit.2', kwargs=dict(n_splits=<(5)many>, test_size=.5, random_state=44), groups='quarter'),
    def func(*, t):
        group = P.quarter[~ho]
        h = frzset(group)
        g = frzset(random.Random(t).sample(h,len(h)//2))
        tr, cv = group.isin(g), group.isin(h-g)
        tr, cv = [F.index[~ho][trcv] for trcv in [tr,cv]]
        return O(Samps=((tr, cv),(cv,tr)), Ctor=(g,h-g))

@inside(LSM, name='Params')
class Params(O()):
    '''parameters constant settings'''
    data = dict(
        objective = 'binary',
        num_iterations = 100000,
        early_stopping_round = 32,
        metric = 'None',
        seed = 44,
        bagging_seed = 45,
        feature_fraction_seed = 46,
    )
# @TEST

pathlib.Path

In [ ]:
LOG = lambda*a,**k: print('>>>', *a, **k)
SEE = print

In [ ]:
mm = ModelManager('/big/data/search/test')
ms = ModelSearch(specs=LSM, mm=mm, X=F, Y=P)
walk = ms.walk()

In [ ]:
for (fi, si), results in walk:
    for a in ['nboost', 'train', 'scores', 'score']:
        SEE(f'{a} = {results[a]}')